# Using LightFM to Create Hybrid Model

As we've seen in our last two modeling sections both models add some value the the recommender system. However, a few issues with the ones before is that content-based doesn't take in mind how others have rated items, only shows us very similar items and therefore we aren't introduced to new items. And with collaborative-fitering we need have a very sparce data set, because most users have not interacted/rated these items. 

So, by combining the two we can overcome these issues. This hybrid method applies both techniques into one.

Next Steps:
1. read in the data and properlly format it
.......

#### Read in the needed libraries:

In [1]:
import pandas as pd
import numpy as np
import spacy
import pickle
from Mod_5_functions import (pickle_file,open_pickle,in_nyc,is_sample,return_lemma,replace_fixed_words)

from lightfm.datasets import fetch_stackexchange

data = fetch_stackexchange('crossvalidated',
                           test_set_fraction=0.1,
                           indicator_features=False,
                           tag_features=True)

train = data['train']
test = data['test']


/Users/elenasm7/anaconda/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


#### Read in the data sets:

In [2]:
from sklearn.metrics.pairwise import cosine_similarity

training_df_cats = open_pickle('Data/training_df_with_categories')

training_df_cats.dropna(inplace=True)
# sklearn.metrics.pairwise.cosine_similarity(X, Y=None, dense_output=True)

In [3]:
training_df_cats.reset_index(drop=True,inplace=True)

In [4]:
training_df_cats.columns

Index(['comapny_source', 'company_loc', 'rev_comp_rating', 'rev_comp_reviews',
       'rev_comp_url', 'rev_company_name', 'userUrl', 'word_count',
       'char_count', 'numerics', 'upper', 'bangs', 'stp_wrd_cnt',
       'rev_comp_reviews_corrections_new', 'new_rating', 'rev_comp_id',
       'categories', 'ids', 'rating', 'review_count'],
      dtype='object')

In [5]:
CF_df = training_df_cats[['userUrl','rev_comp_id','rev_comp_rating']]
CB_df = training_df_cats[['userUrl','rev_comp_id','categories']]

# training_df_cats.columns

In [6]:
from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas(desc="Progress: ")

CB_df.categories = CB_df.categories.progress_apply(lambda cats: ' '.join(cats))

/Users/elenasm7/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [7]:
CB_df.drop_duplicates(inplace=True)

/Users/elenasm7/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
CB_df.reset_index(inplace=True,drop=True)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(0,3))
tfidf_matrix = vec.fit_transform(CB_df.categories)

cosine_similarities = cosine_similarity(tfidf_matrix,Y=None,dense_output=False)

In [10]:
type(cosine_similarities)

scipy.sparse.csr.csr_matrix

In [11]:
# cosine_similarities_2 = cosine_similarity(CF_df,Y=None,dense_output=False)
CF_df.head()

userUrl  \
0  https://www.yelp.com/user_details?userid=exPhu...   
1  https://www.yelp.com/user_details?userid=exPhu...   
2  https://www.yelp.com/user_details?userid=exPhu...   
3  https://www.yelp.com/user_details?userid=exPhu...   
4  https://www.yelp.com/user_details?userid=exPhu...   

                                         rev_comp_id  rev_comp_rating  
0      planet-fitness-manhattan-canal-st-ny-new-york              3.0  
1                         montauk-salt-cave-new-york              2.0  
2  pure-barre-new-york-columbus-circle-60th-and-b...              3.0  
3  return-to-life-center-pilates-and-functional-m...              4.0  
4                                   peloton-new-york              4.0

In [13]:
ratings_df = pd.pivot_table(CF_df, index='userUrl', columns='rev_comp_id', aggfunc=np.max)
ratings_df

rev_comp_rating  \
rev_comp_id                                        14th-street-y-new-york   
userUrl                                                                     
https://www.yelp.com/user_details?userid=-03y31...                    NaN   
https://www.yelp.com/user_details?userid=-0D7SF...                    NaN   
https://www.yelp.com/user_details?userid=-0aZWY...                    NaN   
https://www.yelp.com/user_details?userid=-8SOcH...                    NaN   
https://www.yelp.com/user_details?userid=-9rpsC...                    NaN   
https://www.yelp.com/user_details?userid=-A58lX...                    NaN   
https://www.yelp.com/user_details?userid=-GN5an...                    NaN   
https://www.yelp.com/user_details?userid=-Kh4oI...                    NaN   
https://www.yelp.com/user_details?userid=-L5B5U...                    NaN   
https://www.yelp.com/user_details?userid=-OwQZP...                    NaN   
https://www.yelp.com/user_details?userid=-PsQ4q...                    NaN   
https://www.yelp.com/user_details?userid=-Q-k50...                    NaN   
https://www.yelp.com/user_details?userid=-TDGWI...                    NaN   
https://www.yelp.com/user_details?userid=-TsXE-...                    NaN   
https://www.yelp.com/user_details?userid=-UEpSl...                    NaN   
https://www.yelp.com/user_details?userid=-Wyf4f...                    NaN   
https://www.yelp.com/user_details?userid=-X0nIi...                    NaN   
https://www.yelp.com/user_details?userid=-XdvQI...                    NaN   
https://www.yelp.com/user_details?userid=-Yb1sr...                    NaN   
https://www.yelp.com/user_details?userid=-ZEbvu...                    NaN   
https://www.yelp.com/user_details?userid=-bEaJT...                    NaN   
https://www.yelp.com/user_details?userid=-cCTkU...                    NaN   
https://www.yelp.com/user_details?userid=-dCraL...                    NaN   
https://www.yelp.com/user_details?userid=-dzc9g...                    NaN   
https://www.yelp.com/user_details?userid=-g-CsE...                    NaN   
https://www.yelp.com/user_details?userid=-gdPp4...                    NaN   
https://www.yelp.com/user_details?userid=-kuADe...                    NaN   
https://www.yelp.com/user_details?userid=-lc5T3...                    NaN   
https://www.yelp.com/user_details?userid=-mk3h0...                    NaN   
https://www.yelp.com/user_details?userid=-nd-KE...                    NaN   
...                                                                   ...   
https://www.yelp.com/user_details?userid=zHV4-g...                    NaN   
https://www.yelp.com/user_details?userid=zHvwZn...                    NaN   
https://www.yelp.com/user_details?userid=zJX_dn...                    NaN   
https://www.yelp.com/user_details?userid=zKRIMk...                    NaN   
https://www.yelp.com/user_details?userid=zK__Mo...                    NaN   
https://www.yelp.com/user_details?userid=zL0CQI...                    NaN   
https://www.yelp.com/user_details?userid=zONGOl...                    NaN   
https://www.yelp.com/user_details?userid=zQskJ8...                    NaN   
https://www.yelp.com/user_details?userid=zTTKag...                    NaN   
https://www.yelp.com/user_details?userid=zVAfUd...                    NaN   
https://www.yelp.com/user_details?userid=zViUiV...                    NaN   
https://www.yelp.com/user_details?userid=zWVnSi...                    NaN   
https://www.yelp.com/user_details?userid=zXIBH-...                    NaN   
https://www.yelp.com/user_details?userid=zXY0Nu...                    NaN   
https://www.yelp.com/user_details?userid=zY33UJ...                    NaN   
https://www.yelp.com/user_details?userid=zaLyD0...                    NaN   
https://www.yelp.com/user_details?userid=zeul4M...                    NaN   
https://www.yelp.com/user_details?userid=zgZSv-...                    NaN   
https://www.yelp.com/user_details?userid=zhdqG_...    

In [15]:
import surprise
from surprise import BaselineOnly
from surprise.model_selection import GridSearchCV
from surprise.reader import Reader
from surprise import Dataset

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(CF_df, reader)

param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.02,0.03,0.04],
                             'learning_rate':[0.0055,0.0065,0.01,0.02],
                             'n_epocha':[5,7,10]}}
gs = GridSearchCV(BaselineOnly, param_grid, cv=10)
# ['n_epochs': [5,10,15], 'reg_u': [10,15,20],'reg_i': [5,10,15],
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimati

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimati

In [ ]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

In [ ]:
df_sim_matrix = algo.compute_similarities()

In [ ]:
from scipy.sparse import csr_matrix
df_sim_matrix_sparce = csr_matrix(df_sim_matrix)

In [ ]:
random_state = 42

In [16]:
import pickle
import re
import pandas as pd
import surprise
from surprise import KNNWithMeans
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import NormalPredictor
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise import KNNWithZScore
from surprise import BaselineOnly
from surprise import CoClustering
from surprise import SlopeOne
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.reader import Reader
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from Mod_5_functions import pickle_file,open_pickle

In [17]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


fit_time  test_rmse  test_time
Algorithm                                      
SVDpp            1.737014   1.189098   0.091237
SVD              0.638459   1.204195   0.037358
BaselineOnly     0.058037   1.210912   0.041556
KNNBaseline      0.447860   1.264551   0.145098
KNNBasic         0.687198   1.354196   0.204241
CoClustering     1.095632   1.360526   0.060607
KNNWithZScore    0.823506   1.405585   0.130664
KNNWithMeans     0.387453   1.407419   0.094804
SlopeOne         0.123417   1.426735   0.045398
NMF              1.060548   1.428578   0.028542
NormalPredictor  0.018364   1.708053   0.030757

In [24]:
import numpy as np
np.random.seed(42)

In [21]:
param_grid = {'n_epochs': [10,20,30,35,37], 'lr_all': [0.005,0.006,0.0001],
              'reg_all': [0.4, 0.6,0.2,0.1]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.1675879141840482
{'n_epochs': 35, 'lr_all': 0.006, 'reg_all': 0.2}


In [23]:
param_grid = {'n_epochs': [35,37], 'lr_all': [0.006,0.007,0.008],
              'reg_all': [0.02,0.2,0.25]}
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.1645912771811069
{'n_epochs': 37, 'lr_all': 0.008, 'reg_all': 0.25}


In [25]:
param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.02,0.03,0.04],
                             'learning_rate':[0.0055,0.0065,0.01,0.02],
                             'n_epocha':[5,7,10]}}
gs = GridSearchCV(BaselineOnly, param_grid, cv=5)
# ['n_epochs': [5,10,15], 'reg_u': [10,15,20],'reg_i': [5,10,15],
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimati

In [ ]:
param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.06,0.07,0.08],
                             'learning_rate':[0.095, 0.01,0.015],
                             'n_epocha':[3,4,5,6]}}
gs = GridSearchCV(BaselineOnly, param_grid, cv=10)
# ['n_epochs': [5,10,15], 'reg_u': [10,15,20],'reg_i': [5,10,15],
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])